## Takeaways
- Need to either be able to build conditional formatting in Streamlit, or need to import data as CSV into Google Sheets instead of Excel
    - Will need new Auth key for Google Sheets I think


## App Testing

In [120]:
# Import Libraries
import pandas as pd
import yfinance as yf
import streamlit as st
import os

pwd = os.getcwd()

In [128]:
def ticker_history(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    return ticker_history

In [129]:
def ticker_price_action(ticker_symbol):
    
    ticker_price_action = ticker_history(ticker_symbol)

    # Number of rows displayed
    rows_returned = 5

    # Create derived columns
    ticker_price_action["Ticker"] = ticker_symbol
    ticker_price_action["Wkly_Avg_Close"] = ticker_price_action["Close"].rolling(7).mean()
    ticker_price_action["Wkly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Wkly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Monthly_Avg_Close"] = ticker_price_action["Close"].rolling(30).mean()
    ticker_price_action["Monthly_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Monthly_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Semi_Annual_Avg_Close"] = ticker_price_action["Close"].rolling(183).mean()
    ticker_price_action["Semi_Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Semi_Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Annual_Avg_Close"] = ticker_price_action["Close"].rolling(365).mean()
    ticker_price_action["Annual_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Annual_Avg_Close"]) / ticker_price_action["Close"] * 100
    ticker_price_action["Two_Year_Avg_Close"] = ticker_price_action["Close"].rolling(730).mean()
    ticker_price_action["Two_Year_Avg_Close_Percent"] = (ticker_price_action["Close"] - ticker_price_action["Two_Year_Avg_Close"]) / ticker_price_action["Close"] * 100

    # Drop NAs
    ticker_price_action = ticker_price_action.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_price_action[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

ticker_price_action("PEP")

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Wkly_Avg_Close,Wkly_Avg_Close_Percent,Monthly_Avg_Close,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close,Annual_Avg_Close_Percent,Two_Year_Avg_Close,Two_Year_Avg_Close_Percent
Date,,,,,,,,,,,,,,,,,,
1972-06-01 00:00:00-04:00,0.000000,0.419855,0.415001,0.419855,318600,0.00000,0.0,PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-06-02 00:00:00-04:00,0.419855,0.424102,0.418642,0.418642,140400,0.00000,0.0,PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-06-05 00:00:00-04:00,0.419859,0.420468,0.405864,0.413774,469800,0.00463,0.0,PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-06-06 00:00:00-04:00,0.410124,0.410124,0.408907,0.410124,140400,0.00000,0.0,PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-06-07 00:00:00-04:00,0.410123,0.410732,0.406472,0.407689,178200,0.00000,0.0,PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07 00:00:00-05:00,179.520004,180.619995,178.830002,180.210007,3246100,0.00000,0.0,PEP,179.940000,0.149829,172.169000,4.462020,168.198476,6.665296,161.887601,10.167252,145.702583,19.148450
2022-11-08 00:00:00-05:00,180.440002,182.110001,178.800003,180.710007,3724700,0.00000,0.0,PEP,179.722859,0.546261,172.659001,4.455208,168.295378,6.869917,161.993675,10.357109,145.776415,19.331299
2022-11-09 00:00:00-05:00,179.949997,181.070007,178.399994,178.550003,4723600,0.00000,0.0,PEP,179.290002,-0.414449,172.988334,3.114908,168.378067,5.696968,162.094647,9.216105,145.848416,18.315086


In [115]:
def ticker_financials(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_financials = ticker.quarterly_financials

    # Number of rows displayed
    rows_returned = 5
    
    return ticker_financials

ticker_financials("MCD")

,2022-09-30,2022-06-30,2022-03-31,2021-12-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,2536200000.0,1409100000.0,1541200000.0,2107800000.0
Minority Interest,None,None,None,None
Net Income,1981600000.0,1188000000.0,1104400000.0,1638800000.0
Selling General Administrative,576400000.0,611200000.0,584300000.0,755400000.0
Gross Profit,3446100000.0,3302100000.0,3050100000.0,3261000000.0
Ebit,2776400000.0,2597900000.0,2312600000.0,2419500000.0
Operating Income,2776400000.0,2597900000.0,2312600000.0,2419500000.0
Other Operating Expenses,NaN,NaN,60500000.0,-400000.0


In [111]:
def ticker_function(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    # Number of rows displayed
    rows_returned = 5

    # Create derived columns
    ticker_history["Ticker"] = ticker_symbol
    ticker_history["Wkly_Avg_Close"] = ticker_history["Close"].rolling(7).mean()
    ticker_history["Wkly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Wkly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Monthly_Avg_Close"] = ticker_history["Close"].rolling(30).mean()
    ticker_history["Monthly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Monthly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Semi_Annual_Avg_Close"] = ticker_history["Close"].rolling(183).mean()
    ticker_history["Semi_Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Semi_Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Annual_Avg_Close"] = ticker_history["Close"].rolling(365).mean()
    ticker_history["Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Two_Year_Avg_Close"] = ticker_history["Close"].rolling(730).mean()
    ticker_history["Two_Year_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Two_Year_Avg_Close"]) / ticker_history["Close"] * 100

    # Drop NAs
    ticker_history = ticker_history.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_history[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

# Assign function call to ticker
mcd_ticker = ticker_function("MCD")
# pep_ticker = ticker_function("PEP")
# msft_ticker = ticker_function("MSFT")

# Export to CSV
# mcd_ticker.to_csv(pwd + "\\stocks.csv", mode ="w", header=True, index=False)
# pep_ticker.to_csv(pwd + "\\stocks.csv", mode ="a", header=False, index=False)
# msft_ticker.to_csv(pwd + "\\stocks.csv", mode ="a", header=False, index=False)

# Create CSV data frame the import into Streamlit
# df = pd.read_csv(pwd + "\\stocks.csv")

# Run Streamlit
# st.title("Weekly DCA Assessment")
# st.write(df)

# add_sidebar = st.sidebar.selectbox("Test Side Bar Title", ("Test 1", "Test 2"))



,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Wkly_Avg_Close,Wkly_Avg_Close_Percent,Monthly_Avg_Close,Monthly_Avg_Close_Percent,Semi_Annual_Avg_Close,Semi_Annual_Avg_Close_Percent,Annual_Avg_Close,Annual_Avg_Close_Percent,Two_Year_Avg_Close,Two_Year_Avg_Close_Percent
Date,,,,,,,,,,,,,,,,,,
1969-07-08 00:00:00-04:00,0.000000,0.117130,0.112947,0.113412,1057050,0.0,0.0,MCD,0.113943,-0.468313,0.175804,-55.013591,0.216446,-90.849174,0.244780,-115.832544,0.204647,-80.445316
1969-07-09 00:00:00-04:00,0.000000,0.112947,0.112017,0.112017,1129950,0.0,0.0,MCD,0.113545,-1.363490,0.170986,-52.642004,0.215870,-92.710575,0.244150,-117.956992,0.204634,-82.680362
1969-07-10 00:00:00-04:00,0.000000,0.112018,0.111553,0.111553,656100,0.0,0.0,MCD,0.113279,-1.547388,0.166322,-49.096908,0.215255,-92.961815,0.243506,-118.286759,0.204611,-83.420730
1969-07-11 00:00:00-04:00,0.000000,0.112018,0.111553,0.111553,935550,0.0,0.0,MCD,0.113146,-1.428387,0.161659,-44.916375,0.214622,-92.394878,0.242865,-117.712602,0.204595,-83.406305
1969-07-14 00:00:00-04:00,0.000000,0.111553,0.100397,0.102721,1445850,0.0,0.0,MCD,0.111287,-8.338855,0.156639,-52.488867,0.213954,-108.285996,0.242225,-135.807157,0.204565,-99.145618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07 00:00:00-05:00,275.070007,278.079987,275.070007,276.410004,3029500,0.0,0.0,MCD,273.497144,1.053818,250.553666,9.354342,247.715079,10.381290,244.312992,11.612102,220.599828,20.191084
2022-11-08 00:00:00-05:00,276.859985,279.519989,276.500000,278.399994,2628800,0.0,0.0,MCD,274.051431,1.561984,251.943666,9.502992,247.876063,10.964056,244.452337,12.193842,220.726887,20.715915
2022-11-09 00:00:00-05:00,278.510010,280.250000,277.679993,277.790009,2610800,0.0,0.0,MCD,274.784289,1.082012,253.305333,8.814095,248.045323,10.707615,244.591744,11.950849,220.854858,20.495752


## Scripts that were not yet consolidated into a function

In [105]:
mcd = yf.Ticker("mcd")
mcd_history = mcd.history(period="max")

# Create Ticker Column
mcd_history["Ticker"] = "MCD"

# Create rolling weekly average column
mcd_history["weekly_avg_close"] = mcd_history["Close"].rolling(7).mean()

# Create weekly average close percent column
mcd_history["weekly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["weekly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling monthly average column
mcd_history["monthly_avg_close"] = mcd_history["Close"].rolling(30).mean()

# Create monthly average close percent column
mcd_history["monthly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["monthly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling semi_annual average column
mcd_history["semi_annual_avg_close"] = mcd_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
mcd_history["semi_annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["semi_annual_avg_close"]) / mcd_history["Close"] * 100

# Create rolling annual average column
mcd_history["annual_avg_close"] = mcd_history["Close"].rolling(365).mean()

# Create annual average close percent column
mcd_history["annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["annual_avg_close"]) / mcd_history["Close"] * 100


# Create rolling two_year average column
mcd_history["two_year_avg_close"] = mcd_history["Close"].rolling(730).mean()

# Create two_year average close percent column
mcd_history["two_year_avg_close_percent"] = (mcd_history["Close"] - mcd_history["two_year_avg_close"]) / mcd_history["Close"] * 100

# Drop NAs
mcd_history = mcd_history.dropna()

# Grab only needed columns
mcd_price_action = mcd_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
mcd_price_action = mcd_price_action.reset_index()

# Export to CSV
# mcd_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [106]:
pep = yf.Ticker("PEP")
pep_history = pep.history(period="max")

# Create Ticker Column
pep_history["Ticker"] = "PEP"

# Create rolling weekly average column
pep_history["weekly_avg_close"] = pep_history["Close"].rolling(7).mean()

# Create weekly average close percent column
pep_history["weekly_avg_close_percent"] = (pep_history["Close"] - pep_history["weekly_avg_close"]) / pep_history["Close"] * 100

# Create rolling monthly average column
pep_history["monthly_avg_close"] = pep_history["Close"].rolling(30).mean()

# Create monthly average close percent column
pep_history["monthly_avg_close_percent"] = (pep_history["Close"] - pep_history["monthly_avg_close"]) / pep_history["Close"] * 100

# Create rolling semi_annual average column
pep_history["semi_annual_avg_close"] = pep_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
pep_history["semi_annual_avg_close_percent"] = (pep_history["Close"] - pep_history["semi_annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling annual average column
pep_history["annual_avg_close"] = pep_history["Close"].rolling(365).mean()

# Create annual average close percent column
pep_history["annual_avg_close_percent"] = (pep_history["Close"] - pep_history["annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling two_year average column
pep_history["two_year_avg_close"] = pep_history["Close"].rolling(730).mean()

# Create two_year average close percent column
pep_history["two_year_avg_close_percent"] = (pep_history["Close"] - pep_history["two_year_avg_close"]) / pep_history["Close"] * 100


# Drop NAs
pep_history = pep_history.dropna()

# Grab only needed columns
pep_price_action = pep_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
pep_price_action = pep_price_action.reset_index()


# Export to CSV
# pep_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [107]:
msft = yf.Ticker("MSFT")
msft_history = msft.history(period="max")

msft_history["Ticker"] = "MSFT"

# Create rolling weekly average column
msft_history["weekly_avg_close"] = msft_history["Close"].rolling(7).mean()

# Create weekly average close percent column
msft_history["weekly_avg_close_percent"] = (msft_history["Close"] - msft_history["weekly_avg_close"]) / msft_history["Close"] * 100

# Create rolling monthly average column
msft_history["monthly_avg_close"] = msft_history["Close"].rolling(30).mean()

# Create monthly average close percent column
msft_history["monthly_avg_close_percent"] = (msft_history["Close"] - msft_history["monthly_avg_close"]) / msft_history["Close"] * 100

# Create rolling semi_annual average column
msft_history["semi_annual_avg_close"] = msft_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
msft_history["semi_annual_avg_close_percent"] = (msft_history["Close"] - msft_history["semi_annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling annual average column
msft_history["annual_avg_close"] = msft_history["Close"].rolling(365).mean()

# Create annual average close percent column
msft_history["annual_avg_close_percent"] = (msft_history["Close"] - msft_history["annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling two_year average column
msft_history["two_year_avg_close"] = msft_history["Close"].rolling(730).mean()

# Create two_year average close percent column
msft_history["two_year_avg_close_percent"] = (msft_history["Close"] - msft_history["two_year_avg_close"]) / msft_history["Close"] * 100


# Drop NAs
msft_history = msft_history.dropna()

# Grab only needed columns
msft_price_action = msft_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
msft_price_action = msft_price_action.reset_index()

# Export to CSV
# msft_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)